In [ ]:
import os 
import pickle
import numpy as np
import pandas as pd 
from glob import glob 
from sklearn.metrics import accuracy_score, classification_report
from scipy.spatial.distance import mahalanobis

In [ ]:
def dtw_distance(x, y, dist='euclidean'):
    m, n = len(x), len(y)
    dtw = np.zeros((m+1, n+1))
    
    # Initialize the first row and column of the DTW matrix
    for i in range(1, m+1):
        dtw[i, 0] = float('inf')
    for j in range(1, n+1):
        dtw[0, j] = float('inf')
    dtw[0, 0] = 0
    
    # Compute the DTW matrix
    for i in range(1, m+1):
        for j in range(1, n+1):
            if dist == 'euclidean':
              cost = euclidean_distance(x[i-1], y[j-1])  # Distance measure between elements x[i-1] and y[j-1]
            if dist == 'mahalanobis':
              cost = mahalanobis_distance(x[i-1], y[j-1])
            dtw[i, j] = cost + min(dtw[i-1, j], dtw[i, j-1], dtw[i-1, j-1])
    
    return dtw[m, n]

def euclidean_distance(a, b):
    # Calculate the distance between two elements a and b
    # Modify this function to use an appropriate distance measure for your multivariate time series
    
    # For example, Euclidean distance can be used as follows:
    return np.linalg.norm(a - b)
    
    # Alternatively, you can use other distance measures like Manhattan distance or cosine similarity
    # pass

def mahalanobis_distance(a, b):
    # Calculate the Mahalanobis distance between two elements a and b
    # cov is the covariance matrix
    # cov = np.array([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
    #                 [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
    #                 [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.],
    #                 [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.],
    #                 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.],])
    
    cov = np.array([[1., 1. , 0.5, 0.5, 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [1., 1. , 0.5, 0.5, 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0., 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ,0. , 0. , 0. , 0. , 0. , 0. ,],
                    [0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0.,  0.,  0.,  0.,  0.,  0.,],
                    [0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0.5, 0.5, 0.5, 0.5, 0.,  0.,  0.,  0.,  0.,  0.,],
                    [0., 0., 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 1. , 1. , 0. , 0. ,0.5, 0.5, 0. , 0. , 0. , 0. ,],
                    [0., 0., 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 1. , 1. , 0. , 0. ,0.5, 0.5, 0. , 0. , 0. , 0. ,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. ,0. , 0. , 0.5, 0.5, 0. , 0. ,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 1. , 1. ,0. , 0. , 0.5, 0.5, 0. , 0. ,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ,1. , 1. , 0. , 0. , 0.5, 0.5,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ,1. , 1. , 0. , 0. , 0.5, 0.5,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0.,  0.,  1.,  1.,  0.,  0.,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0.,  0.,  1.,  1.,  0.,  0.,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0.5, 0.5, 0. , 0. , 1. , 1. ,],
                    [0., 0., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,0.5, 0.5, 0. , 0. , 1. , 1. ,]])
    # Calculate the Mahalanobis distance using the scipy.spatial.distance.mahalanobis function
    return mahalanobis(a, b, cov)




In [ ]:
# Example usage
x = np.array([[1, 3, 5], [2, 4, 6], [1, 2, 3]])
y = np.array([[1, 4, 7], [3, 5, 7], [1, 2, 3], [4, 5, 6]])
distance = dtw_distance(x, y)
print("DTW distance:", distance)

DTW distance: 9.164271207775299


In [ ]:
x.shape

(3, 3)

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_root = '/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/epoch_vids/CKIM/MHEALTH/5 28 11 - Test1/'

In [ ]:
with open(f"{data_root}skeleton_label_predictions.pickle", 'rb') as handle:
    pred_skeleton_data = pickle.load(handle)

In [ ]:
with open(f"{data_root}skeleton_original_seq.pickle", 'rb') as handle:
    origin_skeleton_data = pickle.load(handle)

In [ ]:
origin_skeleton_data['Frontal elevation of arms'].shape

(60, 24)

In [ ]:
pred_skeleton_data['label'][:5]

[{'id': 0, 'target': 'Climbing stairs', 'prediction': 'Climbing stairs'},
 {'id': 1, 'target': 'Climbing stairs', 'prediction': 'Climbing stairs'},
 {'id': 2, 'target': 'Cycling', 'prediction': 'Climbing stairs'},
 {'id': 3, 'target': 'Climbing stairs', 'prediction': 'Climbing stairs'},
 {'id': 4, 'target': 'Cycling', 'prediction': 'Climbing stairs'}]

In [ ]:
_, _, files = next(os.walk(f"{data_root}/videos"))


In [ ]:
pred_files, target_files = {}, {}
for f in files:
  id = int(f.split('_')[1])
  if f[-17:] == 'original_skel.mp4':
    target_files[id] = f
  else:
    pred_files[id] = f.split('_m_')[1].split('_')[0]


In [ ]:
files[0]

'sample_0_target_Climbing stairs_original_skel.mp4'

In [ ]:
files[0].split('_')[1]

'0'

In [ ]:
files[0]

'sample_0_target_Climbing stairs_original_skel.mp4'

---

In [ ]:
pred_df = pd.DataFrame().from_records(pred_skeleton_data['label'])
pred_df.head()

,id,target,prediction
0,0,Climbing stairs,Climbing stairs
1,1,Climbing stairs,Climbing stairs
2,2,Cycling,Climbing stairs
3,3,Climbing stairs,Climbing stairs
4,4,Cycling,Climbing stairs


In [ ]:
pred_df['matching'] = pred_df['id'].map(pred_files)
pred_df.head()

,id,target,prediction,matching
0,0,Climbing stairs,Climbing stairs,Knees bending (crouching)
1,1,Climbing stairs,Climbing stairs,Knees bending (crouching)
2,2,Cycling,Climbing stairs,Knees bending (crouching)
3,3,Climbing stairs,Climbing stairs,Walking
4,4,Cycling,Climbing stairs,Knees bending (crouching)


---

In [ ]:
#MEALTH
mhealth_super_dict = {
    'Static activities': ['Standing still', 'Sitting and relaxing', 'Lying down'],
    'Walking activities': ['Walking', 'Climbing stairs', 'Jogging', 'Running'],
    'Sports': ['Waist bends forward', 'Frontal elevation of arms' , 'Knees bending (crouching)', 'Cycling', 'Jump front & back']
}



In [ ]:
# UTD-MHAD
mhad_super_dict = {
    'One-hand activities': ['swipe left', 'swipe right', 'wave', 'throw', 'knock', 'catch'],
    'Two-hand activities': ['clap', 'arm cross', 'arm curl', 'push'],
    'Drawing activities': ['draw x', 'draw circle(clockwise)', 'draw circle(counter clockwise)', 'draw triangle'],
    'Sports': ['basketball shoot', 'boxing', 'baseball swing', 'tennis serve', 'tennis swing'],
    'Activities involving legs': ['bowling', 'pickup & throw', 'jog', 'walk', 'sit to stand', 'stand to sit', 'lunge', 'squat']
}



In [ ]:
# DaLiAc
daliac_super_dict = {
    'Static activities': ['sitting', 'lying', 'standing'],
    'House chores': ['washing dishes', 'vacuuming', 'sweeping'],
    'Walking activities': ['walking', 'ascending stairs', 'descending stairs'],
    'Other activities': ['treadmill running', 'cycling', 'rope jumping']
}


In [ ]:
# PAMAP2
pamap2_super_dict = {
    'Static activities': ['lying', 'sitting', 'standing'],
    'Walking activities': ['walking', 'Nordic walking', 'ascending stairs', 'descending stairs'],
    'House chores': ['vacuum cleaning', 'ironing', 'folding laundry', 'house cleaning'],
    'Sports': ['running', 'cycling', 'playing soccer', 'rope jumping'],
    'Sitting activities': ['watching TV', 'computer work', 'car driving']
}


---

In [ ]:
super_class = {}
for i,j in mhealth_super_dict.items():
    for c in j:
        super_class[c] = i 

In [ ]:
super_class

{'Standing still': 'Static activities',
 'Sitting and relaxing': 'Static activities',
 'Lying down': 'Static activities',
 'Walking': 'Walking activities',
 'Climbing stairs': 'Walking activities',
 'Jogging': 'Walking activities',
 'Running': 'Walking activities',
 'Waist bends forward': 'Sports',
 'Frontal elevation of arms': 'Sports',
 'Knees bending (crouching)': 'Sports',
 'Cycling': 'Sports',
 'Jump front & back': 'Sports'}

In [ ]:
pred_df['super_target'] = pred_df['target'].map(super_class)
pred_df['super_prediction'] = pred_df['prediction'].map(super_class)
pred_df['super_matching'] = pred_df['matching'].map(super_class)
pred_df.head()

,id,target,prediction,matching,super_target,super_prediction,super_matching
0,0,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports
1,1,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports
2,2,Cycling,Climbing stairs,Knees bending (crouching),Sports,Walking activities,Sports
3,3,Climbing stairs,Climbing stairs,Walking,Walking activities,Walking activities,Walking activities
4,4,Cycling,Climbing stairs,Knees bending (crouching),Sports,Walking activities,Sports


In [ ]:
pred_df['correct'] = (pred_df['target'] == pred_df['prediction']).astype(int)
pred_df.head()

,id,target,prediction,matching,super_target,super_prediction,super_matching,correct
0,0,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports,1
1,1,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports,1
2,2,Cycling,Climbing stairs,Knees bending (crouching),Sports,Walking activities,Sports,0
3,3,Climbing stairs,Climbing stairs,Walking,Walking activities,Walking activities,Walking activities,1
4,4,Cycling,Climbing stairs,Knees bending (crouching),Sports,Walking activities,Sports,0


In [ ]:
# super class accuracy 
correct_df = pred_df.loc[pred_df['correct'] == 1, :]
print(accuracy_score(y_true=correct_df['super_prediction'], y_pred=correct_df['super_matching']))
print(classification_report(y_true=correct_df['super_prediction'], y_pred=correct_df['super_matching'], zero_division=0))

0.8055555555555556
                    precision    recall  f1-score   support

            Sports       0.00      0.00      0.00         0
 Static activities       1.00      1.00      1.00        18
Walking activities       1.00      0.61      0.76        18

          accuracy                           0.81        36
         macro avg       0.67      0.54      0.59        36
      weighted avg       1.00      0.81      0.88        36



In [ ]:
# super class tendency - predictiion super class 
incorrect_df = pred_df.loc[pred_df['correct'] == 0, :]
print(accuracy_score(y_true=incorrect_df['super_prediction'][:-1], y_pred=incorrect_df['super_matching'][:-1]))
print(classification_report(y_true=incorrect_df['super_prediction'][:-1], y_pred=incorrect_df['super_matching'][:-1], zero_division=0))

0.0
                    precision    recall  f1-score   support

            Sports       0.00      0.00      0.00       0.0
Walking activities       0.00      0.00      0.00      27.0

          accuracy                           0.00      27.0
         macro avg       0.00      0.00      0.00      27.0
      weighted avg       0.00      0.00      0.00      27.0



In [ ]:
# super class tendency - predictiion super class 
incorrect_df = pred_df.loc[pred_df['correct'] == 0, :]
print(accuracy_score(y_true=incorrect_df['super_target'][:-1], y_pred=incorrect_df['super_matching'][:-1]))
print(classification_report(y_true=incorrect_df['super_target'][:-1], y_pred=incorrect_df['super_matching'][:-1], zero_division=0))

1.0
              precision    recall  f1-score   support

      Sports       1.00      1.00      1.00        27

    accuracy                           1.00        27
   macro avg       1.00      1.00      1.00        27
weighted avg       1.00      1.00      1.00        27



In [ ]:
# overall interpretability 
print(accuracy_score(y_true=pred_df['super_prediction'][:-1], y_pred=pred_df['super_matching'][:-1]))
print(classification_report(y_true=pred_df['super_prediction'][:-1], y_pred=pred_df['super_matching'][:-1], zero_division=0))

0.4603174603174603
                    precision    recall  f1-score   support

            Sports       0.00      0.00      0.00         0
 Static activities       1.00      1.00      1.00        18
Walking activities       1.00      0.24      0.39        45

          accuracy                           0.46        63
         macro avg       0.67      0.41      0.46        63
      weighted avg       1.00      0.46      0.57        63



---

In [ ]:
pred_skeleton_data['data'].shape

(192, 60, 24)

In [ ]:
sample_pred = pred_df.loc[0, :].to_dict()
sample_pred

{'id': 0,
 'target': 'Climbing stairs',
 'prediction': 'Climbing stairs',
 'matching': 'Knees bending (crouching)',
 'super_target': 'Walking activities',
 'super_prediction': 'Walking activities',
 'super_matching': 'Sports',
 'correct': 1}

In [ ]:
dtw_distance(origin_skeleton_data[sample_pred['matching']], pred_skeleton_data['data'][0])

4.830772537754304

In [ ]:
origin_skeleton_data['matching']

[]

In [ ]:
origin_skeleton_data[sample_pred['matching']].shape

(60, 24)

In [ ]:
pred_skeleton_data['data'][0].shape

(60, 24)

In [ ]:
def apply_func(r):
  i = r['id']
  return dtw_distance(origin_skeleton_data[r['matching']], pred_skeleton_data['data'][i])

In [ ]:
pred_df['dtw_distance'] = pred_df.apply(lambda r: apply_func(r), axis=1)
pred_df.head()

,id,target,prediction,matching,super_target,super_prediction,super_matching,correct,dtw_distance
0,0,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports,1,4.830773
1,1,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports,1,5.319714
2,2,Cycling,Climbing stairs,Knees bending (crouching),Sports,Walking activities,Sports,0,6.858545
3,3,Climbing stairs,Climbing stairs,Walking,Walking activities,Walking activities,Walking activities,1,2.094390
4,4,Cycling,Climbing stairs,Knees bending (crouching),Sports,Walking activities,Sports,0,5.550426


In [ ]:
def apply_func(r):
  i = r['id']
  return dtw_distance(origin_skeleton_data[r['matching']], pred_skeleton_data['data'][i], dist='mahalanobis')

pred_df['mahalanobis_distance'] = pred_df.apply(lambda r: apply_func(r), axis=1)
pred_df.head()

,id,target,prediction,matching,super_target,super_prediction,super_matching,correct,dtw_distance,mahalanobis_distance
0,0,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports,1,4.830773,4.983109
1,1,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports,1,5.319714,5.539813
2,2,Cycling,Climbing stairs,Knees bending (crouching),Sports,Walking activities,Sports,0,6.858545,7.514340
3,3,Climbing stairs,Climbing stairs,Walking,Walking activities,Walking activities,Walking activities,1,2.094390,2.553820
4,4,Cycling,Climbing stairs,Knees bending (crouching),Sports,Walking activities,Sports,0,5.550426,6.127683


In [ ]:
pred_df.loc[pred_df['correct'] == 0, :]['dtw_distance'][:-1].mean(), pred_df.loc[pred_df['correct'] == 0, :]['mahalanobis_distance'][:-1].mean()

(5.9395385373046015, 6.417901606395531)

In [ ]:
pred_df.loc[pred_df['correct'] == 0, :]['dtw_distance'][:-1].std(), pred_df.loc[pred_df['correct'] == 0, :]['mahalanobis_distance'][:-1].std()

(0.44897843557831524, 0.5638781648982669)

In [ ]:
pred_df.loc[pred_df['correct'] == 1, :]['dtw_distance'].mean(), pred_df.loc[pred_df['correct'] == 1, :]['mahalanobis_distance'].mean()

(10.609194462662519, 7.422878826915165)

In [ ]:
pred_df.loc[pred_df['correct'] == 1, :]['dtw_distance'].mean(), pred_df.loc[pred_df['correct'] == 1, :]['mahalanobis_distance'].mean()

(15.94898831016998, 7.34329882584116)

7.3017796825826125

In [ ]:
pred_df.loc[pred_df['correct'] == 1, :]

,id,target,prediction,matching,super_target,super_prediction,super_matching,correct,dtw_distance,mahalanobis_distance
0,0,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports,1,4.830773,5.638457
1,1,Climbing stairs,Climbing stairs,Knees bending (crouching),Walking activities,Walking activities,Sports,1,5.319714,6.300050
3,3,Climbing stairs,Climbing stairs,Walking,Walking activities,Walking activities,Walking activities,1,2.094390,2.818938
5,5,Climbing stairs,Climbing stairs,Walking,Walking activities,Walking activities,Walking activities,1,2.200007,3.001176
10,10,Climbing stairs,Climbing stairs,Running,Walking activities,Walking activities,Walking activities,1,13.215605,19.331822
11,11,Standing still,Standing still,Sitting and relaxing,Static activities,Static activities,Static activities,1,2.328343,3.018884
12,12,Standing still,Standing still,Sitting and relaxing,Static activities,Static activities,Static activities,1,2.118017,2.355160
13,13,Standing still,Standing still,Sitting and relaxing,Static activities,Static activities,Static activities,1,2.046729,2.569257
15,15,Climbing stairs,Climbing stairs,Walking,Walking activities,Walking activities,Walking activities,1,2.261196,3.295455
18,18,Climbing stairs,Climbing stairs,Walking,Walking activities,Walking activities,Walking activities,1,2.950940,4.359745
